# Classification and Change Detection Task

This notebook implements the task to find Sentinel-2 images within the period 2017-2019 with no cloud cover for a specific area of interest.

## Setup and Installation

Install required packages and import necessary libraries.

In [31]:
# Install required packages
!pip install rasterio
!pip install --extra-index-url https://artifactory.vgt.vito.be/api/pypi/python-packages/simple terracatalogueclient
!pip install pyproj
!pip install shapely
!pip install matplotlib
!pip install numpy
!pip install scikit-image

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\cdlie\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://artifactory.vgt.vito.be/api/pypi/python-packages/simple



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\cdlie\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable


...


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\cdlie\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\cdlie\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\cdlie\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [32]:
# Import required libraries
from terracatalogueclient import Catalogue
import datetime as dt
import rasterio
import matplotlib.pyplot as plt
import numpy as np
from skimage import exposure
from pyproj import Transformer, CRS
from shapely.geometry import Polygon

# Initialize the Terrascope catalogue
catalogue = Catalogue()

## Step 1: Find Sentinel-2 Images (2017-2019) with No Cloud Cover

Search for all Sentinel-2 images in the specified area and time period with zero cloud cover.

In [33]:
# Define the date range for 2017-2019
startDate = dt.date(2017, 1, 1)
endDate = dt.date(2019, 12, 31)

# Define the coordinates from the task (WGS84)
lat = 50.379123
lon = 4.392829

# Define a bounding box (adjust size as needed for "appropriate size")
# Using +/- 0.1 degrees (~11 km at this latitude)
bbox = [lon - 0.1, lat - 0.1, lon + 0.1, lat + 0.1]

# Define cloud cover threshold for filtering
cloudcover_threshold = 5

print(f"Searching for products from {startDate} to {endDate}")
print(f"Area of Interest: lat={lat}, lon={lon}")
print(f"Bounding box: {bbox}")

# Get all products in the date range and location
products = list(catalogue.get_products(
    "urn:eop:VITO:TERRASCOPE_S2_TOC_V2",
    start=startDate,
    end=endDate,
    bbox=bbox
))

print(f"\nTotal products found: {len(products)}")

# Filter for images with no cloud cover (cloudCover == 0)
# Note: You may need to adjust this threshold if no images with 0% cloud cover exist
no_cloud_products = [
    product for product in products 
    if product.properties["productInformation"]["cloudCover"] <= cloudcover_threshold
]

# Sort products by cloud cover percentage (ascending)
no_cloud_products.sort(key=lambda p: p.properties["productInformation"]["cloudCover"])

print(f"Products with < {cloudcover_threshold}% cloud cover: {len(no_cloud_products)}")

# Display the cloud-free images
if len(no_cloud_products) > 0:
    print("\nCloud-free products (sorted by cloud cover %):")
    for i, product in enumerate(no_cloud_products, 1):
        print(f"{i}. {product.title}: {product.properties['productInformation']['cloudCover']}% clouds")
else:
    print("\nNo products found.")

Searching for products from 2017-01-01 to 2019-12-31
Area of Interest: lat=50.379123, lon=4.392829
Bounding box: [4.292829, 50.279123, 4.4928289999999995, 50.479123]

Total products found: 1584
Products with < 5% cloud cover: 258

Cloud-free products (sorted by cloud cover %):
1. S2A_20181117T105321_31UFS_TOC_V210: 0.0% clouds
2. S2A_20190215T105131_31UFS_TOC_V210: 0.0% clouds
3. S2B_20181010T104019_31UFS_TOC_V210: 0.0% clouds
4. S2B_20190227T104019_31UFS_TOC_V210: 0.0% clouds
5. S2B_20180506T105029_31UFS_TOC_V210: 0.001% clouds
6. S2B_20180702T104019_31UFS_TOC_V210: 0.001% clouds
7. S2B_20190421T105039_31UFS_TOC_V210: 0.001% clouds
8. S2B_20190627T104029_31UFS_TOC_V210: 0.001% clouds
9. S2A_20190824T105031_31UFS_TOC_V210: 0.002% clouds
10. S2A_20180918T105021_31UFS_TOC_V210: 0.003% clouds
11. S2B_20180222T104029_31UFS_TOC_V210: 0.003% clouds
12. S2B_20180225T105019_31UFS_TOC_V210: 0.006% clouds
13. S2B_20181010T104019_31UER_TOC_V200: 0.007% clouds
14. S2A_20180806T104021_31UFS_TOC_V21

In [ ]:
# Obtain an access token using OIDC password grant
import os
import requests
from pathlib import Path

# Load credentials from .env file if it exists
env_path = Path(".env")
if env_path.exists():
    with open(env_path) as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith('#') and '=' in line:
                key, value = line.split('=', 1)
                os.environ[key.strip()] = value.strip()

token_url = "https://sso.terrascope.be/auth/realms/terrascope/protocol/openid-connect/token"

# Get credentials from environment variables
USERNAME = os.getenv("TERRASCOPE_USERNAME")
PASSWORD = os.getenv("TERRASCOPE_PASSWORD")

if not USERNAME or not PASSWORD:
    print("Error: Set TERRASCOPE_USERNAME and TERRASCOPE_PASSWORD in .env file or environment variables.")
    ACCESS_TOKEN = None
else:
    data = {
        "grant_type": "password",
        "client_id": "public",
        "username": USERNAME,
        "password": PASSWORD,
    }

    try:
        resp = requests.post(token_url, data=data, timeout=30)
        resp.raise_for_status()
        token_payload = resp.json()
        ACCESS_TOKEN = token_payload["access_token"]
        print("Token acquired. Expires in:", token_payload.get("expires_in"), "seconds")
    except Exception as e:
        ACCESS_TOKEN = None
        print("Failed to obtain token:", e)

Token acquired. Expires in: 300 seconds


In [ ]:
# Download required bands using the Bearer token
from pathlib import Path

if ACCESS_TOKEN is None:
    print("No token available; cannot download protected resources.")
else:
    if len(no_cloud_products) == 0:
        print("No products available.")
    else:
        product = no_cloud_products[0]
        print(f"Downloading from: {product.title}")

        session = requests.Session()
        session.headers.update({"Authorization": f"Bearer {ACCESS_TOKEN}"})

        download_dir = Path("sentinel_data")
        download_dir.mkdir(exist_ok=True)

        required_patterns = ['B02_10M', 'B03_10M', 'B04_10M', 'B08_10M', 'B11_20M']
        downloaded = 0

        print("\nDownloading\n")
        for data_url in product.data:
            url = data_url.href
            filename = url.split('/')[-1]

            if downloaded >= 10:
                print("Reached download limit of 10 files.")
                break

            if any(pat in filename for pat in required_patterns):
                local_path = download_dir / filename

                if local_path.exists():
                    print(f"  [Skip] {filename}")
                    downloaded += 1
                    continue

                print(f"  Downloading {filename} ...", end="")
                try:
                    with session.get(url, stream=True, timeout=90) as r:
                        r.raise_for_status()
                        with open(local_path, "wb") as f:
                            for chunk in r.iter_content(chunk_size=8192):
                                if chunk:
                                    f.write(chunk)
                    print(" [Success]")
                    downloaded += 1
                except Exception as e:
                    print(f" [Failed] → {e}")

        print(f"\nDone! {downloaded} files downloaded.")


Downloading


Done! 5 files downloaded.
 [Success]

Done! 5 files downloaded.
